In [59]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sym
from scipy import linalg

### Controller Design (with vx, vy, vz)


In [60]:
# Mass
m = 0.046

# Principle moments of inertia
J_x = 1.57e-05
J_y = 1.60e-05
J_z = 2.05e-05

# Acceleration of gravity
g = 9.81

p_eq = [m, J_x, J_y, J_z]

In [61]:
def Rotate(alpha, beta, gamma):
    Rz = sym.Matrix([[sym.cos(alpha), -sym.sin(alpha), 0],
                 [sym.sin(alpha), sym.cos(alpha), 0],
                 [0, 0, 1]])

    Ry = sym.Matrix([[sym.cos(beta), 0, sym.sin(beta)],
                 [0, 1, 0],
                 [-sym.sin(beta), 0, sym.cos(beta)]])

    Rx = sym.Matrix([[1, 0, 0],
                 [0, sym.cos(gamma), -sym.sin(gamma)],
                 [0, sym.sin(gamma), sym.cos(gamma)]])

    R_VtoO = Rz * Ry * Rx

    return R_VtoO

def N_func(alpha, beta, gamma):
    Ninv = sym.Matrix([[sym.cos(beta)*sym.cos(gamma), -sym.sin(gamma), 0],
                       [sym.cos(beta)*sym.sin(gamma), sym.cos(gamma), 0],
                       [-sym.sin(beta), 0, 1]])

    # N = sym.simplify(Ninv.inv())
    N = Ninv.inv()
    return N

In [62]:
# Added vx,vy,vz as part of state variables
o_x, o_y, o_z, vx, vy, vz, alpha, beta, gamma, wx, wy, wz, ox_dot, oy_dot, oz_dot= sym.symbols(r'o_x o_y o_z vx vy vz \alpha \beta \gamma w_x w_y w_z ox_{dot} oy_{dot} oz_{dot}')
tau_x, tau_y, tau_z, f_z = sym.symbols('tau_x, tau_y, tau_z, f_z')
m, Jx, Jy, Jz = sym.symbols('m J_x J_y J_z')
J_in1 = sym.diag(Jx, Jy, Jz)
tau_in1 = sym.Matrix([tau_x, tau_y, tau_z])
w_01in1 = sym.Matrix([wx, wy, wz])
# state: [o_x, o_x_dot, o_y, o_y_dot, o_z, o_z_dot, alpha, beta, gamma, wx, wy, wz]

# state: [o_x, vx, o_y, vy, o_z, vz, alpha, beta, gamma, wx, wy, wz]
# state_dot: [o_x_dot, vx_dot, o_y_dot, vy_dot, o_z_dot, vz_dot, alpha_dot, beta_dot, gamma_dot, wx_dot, wy_dot, wz_dot]

# inputs: [tau_x, tau_y, tau_z, f_z]
Rotation_matrix = Rotate(alpha, beta, gamma)
N = N_func(alpha, beta, gamma)
# second_derivatives = Rotation_matrix * sym.Matrix([0, 0, f_z])/m + sym.Matrix([0, 0, -g])
angle_derivatives = N * sym.Matrix([wx, wy, wz]) # [gamma_dot, beta_dot, alpha_dot]
w_derivatives = J_in1.inv() * (tau_in1 - w_01in1.cross(J_in1 * w_01in1))
f_in1 = Rotation_matrix.T * sym.Matrix([0, 0, -m * g]) + sym.Matrix([0, 0, f_z])
f_sym = sym.zeros(12,1)
# # f_sym[0] = ox_dot
# f_sym[1] = second_derivatives[0]
# # f_sym[2] = oy_dot
# f_sym[3] = second_derivatives[1]
# # f_sym[4] = oz_dot
# f_sym[5] = second_derivatives[2]
f_sym[6] = angle_derivatives[2]
f_sym[7] = angle_derivatives[1]
f_sym[8] = angle_derivatives[0]
f_sym[9] = w_derivatives[0]
f_sym[10] = w_derivatives[1]
f_sym[11] = w_derivatives[2]

# NEW CHANGES

v_01in1 = sym.Matrix([vx, vy, vz])
o_dot = Rotation_matrix @ v_01in1 # o_dot matrix in terms of vx,vy,vz
v_dot = (1 / m) * (f_in1 - w_01in1.cross(m * v_01in1))
f_sym[0] = o_dot[0]
f_sym[1] = v_dot[0]
f_sym[2] = o_dot[1]
f_sym[3] = v_dot[1]
f_sym[4] = o_dot[2]
f_sym[5] = v_dot[2]

f_sym

Matrix([
[           vx*cos(\alpha)*cos(\beta) + vy*(-sin(\alpha)*cos(\gamma) + sin(\beta)*sin(\gamma)*cos(\alpha)) + vz*(sin(\alpha)*sin(\gamma) + sin(\beta)*cos(\alpha)*cos(\gamma))],
[                                                                                                                                  (m*vy*w_z - m*vz*w_y + 9.81*m*sin(\beta))/m],
[            vx*sin(\alpha)*cos(\beta) + vy*(sin(\alpha)*sin(\beta)*sin(\gamma) + cos(\alpha)*cos(\gamma)) + vz*(sin(\alpha)*sin(\beta)*cos(\gamma) - sin(\gamma)*cos(\alpha))],
[                                                                                                                     (-m*vx*w_z + m*vz*w_x - 9.81*m*sin(\gamma)*cos(\beta))/m],
[                                                                                                       -vx*sin(\beta) + vy*sin(\gamma)*cos(\beta) + vz*cos(\beta)*cos(\gamma)],
[                                                                                                         

In [63]:
s = [o_x, vx, o_y, vy, o_z, vz, alpha, beta, gamma, wx, wy, wz]
i = [tau_x, tau_y, tau_z, f_z]
p = [m, Jx, Jy, Jz]
s_with_des = [o_x, o_y, o_z]

In [64]:
f = sym.lambdify(s + i + p, f_sym)
f

<function _lambdifygenerated(o_x, vx, o_y, vy, o_z, vz, Dummy_174, Dummy_173, _Dummy_172, w_x, w_y, w_z, tau_x, tau_y, tau_z, f_z, m, J_x, J_y, J_z)>

In [65]:
# Mass
m = 0.046

# Principle moments of inertia
J_x = 1.57e-05
J_y = 1.60e-05
J_z = 2.05e-05

# Acceleration of gravity
l_pen = 320e-3 # m

p_eq = [m, J_x, J_y, J_z]

In [66]:
s_eq = [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]
i_eq = [0., 0., 0., g*(m)]

In [67]:
print(f(*s_eq, *i_eq, *p_eq))

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]


In [68]:
A_sym = f_sym.jacobian(s)
B_sym = f_sym.jacobian(i)

A_num = sym.lambdify(s + i + p, A_sym)
B_num = sym.lambdify(s + i + p, B_sym)

A = A_num(*s_eq, *i_eq, *p_eq)
B = B_num(*s_eq, *i_eq, *p_eq)

In [69]:
# state: [o_x, o_x_dot, o_y, o_y_dot, o_z, o_z_dot, alpha, beta, gamma, wx, wy, wz, r, s, rdot, sdot]
############# JANK SHIT    ##############################
A_mod = np.vstack((np.hstack((A, np.zeros((12, 4)))), np.zeros((4,16))))
A_mod[12,14] = 1.0
A_mod[13,15] = 1.0
A_mod[14,12] = g/l_pen
A_mod[14, 7] = -g
A_mod[15,13] = g/l_pen
A_mod[15, 8] = g

B_mod = np.vstack((B, np.zeros((4,4))))

In [70]:

A_str = np.array2string(A_mod,
                        formatter={'float_kind': lambda x: f'{x:5.2f}'},
                        prefix='    ',
                        max_line_width=np.inf)

print(f'A = {A_str}')

A = [[ 0.00  1.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00 -0.00  0.00  9.81  0.00  0.00 -0.00  0.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  1.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00]
     [ 0.00 -0.00  0.00  0.00  0.00  0.00  0.00  0.00 -9.81  0.00  0.00 -0.00  0.00  0.00  0.00  0.00]
     [ 0.00 -0.00  0.00  0.00  0.00  1.00  0.00 -0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00 -0.00  0.00  0.00  0.00  0.00  0.00 -0.00  0.00  0.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  1.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00 -0.00 -0.00  1.00  0.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  1.00  0.00  0.00  0.00  0.00  0.00  0.00]
     [ 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  

In [71]:
B_str = np.array2string(B_mod,
                        formatter={'float_kind': lambda x: f'{x:10.2f}'},
                        prefix='    ',
                        max_line_width=np.inf)
B
print(f'B = {B_str}')

B = [[      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00      21.74]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [  63694.27       0.00       0.00       0.00]
     [      0.00   62500.00       0.00       0.00]
     [      0.00       0.00   48780.49       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]
     [      0.00       0.00       0.00       0.00]]


In [74]:
Q = np.diag([
    600., # o_x
    100., # o_x_dot
    600., # o_y
    100., # o_y_dot
    1., # o_z
    1., # o_z_dot
    1., # alpha
    5., # beta
    5., # gamma
    1., # wx
    1., # wy
    1., # wz
    20., # r
    20., # s
    1., # rdot
    1.  # sdot
])

R = np.diag([
    3 * 1E7, #tau_x
    3 * 1E7,#tau_y
    3 * 1E7,
    3 * 100,
])

def lqr(A, B, Q, R):
    P = linalg.solve_continuous_are(A, B, Q, R)
    K = linalg.inv(R) @  B.T @ P
    return K


In [75]:
K = lqr(A_mod, B_mod, Q, R)

K

array([[-6.17737039e-14, -4.95042886e-14,  4.47213595e-03,
         4.36930478e-03,  4.14156508e-17,  6.09227844e-17,
        -1.42486209e-16,  3.99161053e-14,  1.03152527e-02,
         5.97689106e-04,  6.59116446e-16,  6.99584345e-17,
        -6.07893048e-13,  8.62435385e-02, -1.09694080e-13,
         1.55767705e-02],
       [-4.47213595e-03, -4.37351999e-03, -1.18548340e-14,
        -3.26871538e-16,  1.34704402e-17,  4.38367374e-17,
         6.10320278e-16,  1.04060895e-02,  1.51976715e-14,
         6.46758013e-16,  6.05250526e-04, -1.79386450e-16,
        -8.66537157e-02,  8.80865500e-14, -1.56508507e-02,
         1.59578306e-14],
       [ 5.68858356e-15,  5.02552383e-15,  1.08573681e-14,
         5.19714343e-15,  2.53564216e-18,  7.95018271e-18,
         1.82574186e-04, -4.40944014e-15,  2.28408224e-15,
         5.35779230e-17, -1.40008937e-16,  2.02036816e-04,
         6.37036375e-14,  5.05120514e-14,  1.14800316e-14,
         9.12282157e-15],
       [-8.21623337e-14, -6.52389654e

In [56]:
r, rdot, s, sdot = sym.symbols('r, rdot, s, sdot')
real_s = [o_x, vx, o_y, vy, o_z, vz, alpha, beta, gamma, wx, wy, wz, r, s, rdot, sdot]

In [57]:
def export_controller(K, s, i, s_with_des, i_eq,
                      decimals=8,
                      suffix='',
                      line_ending=''):
    """
    K is a gain matrix, of size m x n
    s is a list of states as symbolic variables, of length n
    i is a list of inputs as symbolic variables, of length m
    s_with_des is a list of states that have desired values, as
        symbolic variables - if there are no such states, then
        this should be an empty list []
    i_eq is a list of equilibrium values of inputs, of length m
    decimals is the number of decimals to include when printing
        each value
    suffix is the character (if any) to print after each number,
        for example 'f' to indicate a "float" when exporting to C
    line_ending is the character (if any) to print after each
        line, for example ';' when exporting to C
    """
    
    s_name = [scur.name for scur in s]
    s_name[6] = 'psi'
    s_name[7] = 'theta'
    s_name[8] = 'phi'
    i_name = [icur.name for icur in i]
    for row in range(len(i_name)):
        input_string = ''
        for col in range(len(s_name)):
            k = K[row, col]
            if not np.isclose(k, 0.):
                if (k < 0) and input_string:
                    input_string += ' +'
                if s[col] in s_with_des:
                    n = f'({s_name[col]} - {s_name[col]}_des)'
                else:
                    n = s_name[col]
                input_string += f' {-k:.{decimals}f}{suffix} * {n}'
        if not np.isclose(i_eq[row], 0.):
            if (i_eq[row] > 0) and input_string:
                input_string += ' +'
            input_string += f' {i_eq[row]:.{decimals}f}{suffix}'
        print(f'{i_name[row]} ={input_string}{line_ending}')


# # For python simulation
# export_controller(
#     K,               # the gain matrix
#     s,               # list of states as symbolic variables
#     i,               # list of inputs as symbolic variables
#     s_with_des,      # list of states that have desired values as symbolic variables
#     i_eq,            # list of equilibrium values of inputs
# )

# For C
export_controller(
    K,               # the gain matrix
    real_s,               # list of states as symbolic variables
    i,               # list of inputs as symbolic variables
    s_with_des,      # list of states that have desired values as symbolic variables
    i_eq,            # list of equilibrium values of inputs
    suffix='f',      # character to print after each number (indicates a "float")
    line_ending=';'  # character to print after each line
)


tau_x = -0.00447214f * (o_y - o_y_des) -0.00436930f * vy -0.01031525f * phi -0.00059769f * w_x -0.08624354f * s -0.01557677f * sdot;
tau_y = 0.00447214f * (o_x - o_x_des) + 0.00437352f * vx -0.01040609f * theta -0.00060525f * w_y + 0.08665372f * r + 0.01565085f * rdot;
tau_z = -0.00018257f * psi -0.00020204f * w_z;
f_z = -0.05773503f * (o_z - o_z_des) -0.08520161f * vz + 0.33354000f;
